In [6]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import re

In [7]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

page = requests.get("https://www.espn.com/mlb/history/leaders/_/breakdown/season/year/2025", headers=headers)
soup = BeautifulSoup(page.text, 'html.parser')

In [13]:
# Create column names from table header
## Identify header row
header = soup.find('tr', attrs={'class':'colhead'})
## Extract column names from header row
column_names = [col.get_text() for col in header.find_all('td')]
## Create an empty dataframe with column names
df = pd.DataFrame(columns=column_names)
df


,,PLAYER,YRS,G,AB,R,H,2B,3B,HR,RBI,BB,SO,SB,CS,BA


In [14]:
for i in range(1, 326, 50):
    url = f'https://www.espn.com/mlb/history/leaders/_/breakdown/season/year/2025/start/{i}'
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')

    players = soup.find_all('tr', attrs={'class':re.compile('row player-10-')})
    for player in players:
        stats = [stat.get_text() for stat in player.find_all('td')]
        temp_df = pd.DataFrame(stats).transpose()
        temp_df.columns = column_names
        df = pd.concat([df, temp_df], ignore_index=True)

In [15]:
df

,,PLAYER,YRS,G,AB,R,H,2B,3B,HR,RBI,BB,SO,SB,CS,BA
0,1,Aaron Judge,10,152,541,137,179,30,2,53,114,124,160,12,5,.331
1,2,Miguel Andujar,9,94,321,36,102,17,1,10,44,17,49,1,0,.318
2,3,Jonathan Aranda,4,106,370,56,117,22,0,14,59,41,107,0,2,.316
3,4,Bo Bichette,7,139,582,78,181,44,1,18,94,40,91,4,3,.311
4,,Jacob Wilson,2,125,486,62,151,26,0,13,63,27,39,5,2,.311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,321,Joc Pederson,12,96,265,28,48,10,1,9,26,34,65,2,0,.181
321,322,LaMonte Wade Jr.,7,80,209,19,35,8,2,2,18,27,58,1,0,.167
322,,Henry Davis,3,87,252,25,42,7,0,7,22,18,76,2,1,.167
323,324,Oswald Peraza,4,106,244,24,40,6,0,5,20,17,80,9,0,.164


In [9]:
# Write df to csv
df.to_csv('mlb_leaders_2025.csv', index=False)